# Grafos

Antes de explicar y desarrollar algoritmos específicos, debemos comentar un poco las diferentes estructuras y algunas decisiones de implementación realizadas, así como implementar un par de generadores de grafos.

## Limitaciones

* Los códigos que vamos a desarrollar son explicativos, prima la claridad y la exposición de algunas limitaciones sobre la versatilidad.
* Existen librerías para grafos en Julia (https://github.com/JuliaGraphs/LightGraphs.jl y https://github.com/JuliaArchive/Graphs.jl entre otras).
* Reimplementaremos muchas cosas. Es el precio a pagar por intentar explicar algoritmos y no hacer una librería de grafos.
* Trataremos un grafo no dirigido como un grafo dirigido con los arcos. Esto es una solución válida para los problemas y algoritmos que veremos aquí pero muy discutible para otros problemas (como Spanning Tree)

## Tipos de grafos

Veremos tres tipos principales de grafos

* Grafos densos. En un grafo denso (ratio grande de arcos / aristas) es conveniente guardar la informaión de los arcos a través de una matriz.
* Grafos "sparse" tipo (I). Son los grafos en que analizaremos arcos pensando en su vértice origen. De forma similar podríamos discutir aquellos grafos en que los arcos se analizan pensando en su vértice destino.
* Grafos "sparse" tipo (II). Queremos referenciarnos a un arco tanto por su origen como por su destino.

## Algoritmos analizados

Veremos los siguientes problemas y algoritmos:

* Caminos extremos desde un vértice origen
    - Dijkstra
    - Bellman-Ford
* Caminos extremos entre toda pareja de vértices
    - Floyd-Warshall
* Problemas de flujos máximos
    - Ford-Fulkerson
    - Edmonds-Karp
    - Dinic
    - Goldberg-Tarjan (Push-relabel)
* Problemas de flujos máximos a costo mínimo
    - Minimum mean cycle-cancelling
    - Successive shortest path
    - Capacity Scaling
    - Orlin

## Otros

Por primera vez vamos a llamar otro archivo desde uno de nuestros libros. El código empieza a ser demasiado grande como para usar una hoja de Jupyter

## Referencias

Hay mucha literatura sobre el tema:

* Posiblemente el libro más completo es "Network Flows: Theory, Algorithms, and Applications" de Ahuja, Magnanti y Orlin.
* Alternativamente y si no se ne esita trabajar con costes "Introduction to algorithms" de Cormen, Leiserson, Rivest y Stein es bastante más claro, menos enciclopédico y, a mi entender, contiene más información que hacen de él un muy buen libro de referencia.
* Finalmente, yo uso mucho "Combinatorial Optimization. Theory and Algorithms" de Korte y Vygen. El material de estas clases se basa principalmente en este tercer libro.

# Grafo

Como mínimo, un grafo queda definido por su número de vértices, y por la existencia de un arco entre pareja de vértices. 

Nota: esta definición es incompleta y directamente falsa, pero será la que usemos en el tema. Estamos descartando entre otros, grafos en los que hay arcos que unen varios vértices (más de dos) y grafos en que hay más de un arco o arista uniendo una pareja de vértices

In [24]:
struct grafoSimple
    n::Int64
    arco::Array{Bool,2} #arco[i,j]=true indica que existe un arco entre i y j 
end

function generarGrafoSimple(n::Int64,sp::Float64)
    aristas=Array{Bool}(n,n)
    for i in 1:n
        for j in 1:n
            if rand()<sp
                aristas[i,j]=true
            else
                aristas[i,j]=false
            end
        end
    end
    return 0 #grafoSimple(n,aristas) 
end


generarGrafoSimple(5,0.2)

LoadError: syntax: extra token "grafoSimple" after end of expression

Este tipo de grafos ya nos nos permitirá probar dos ideas generales en grafos. La primera es la exploración primero en profundidad (depth-first) y la segunda es la exploración primero en anchura (breadth-first). 

Usaremos ambos métodos para testear conectividad.

In [19]:
function conectividadDepthFirst(G::grafoSimple,origen::Int64)
    return 0
end

function conectividadBreadthFirst(G::grafoSimple,origen::Int64)
    return 0
end

srand(0)
grafo = generarGrafoSimple(n=5,sp=0.25)
println("grafo: ",grafo)
conectividadDepthFirst(grafo,1)
conectividadBreadthFirst(grafo,1)

LoadError: UndefVarError: generarGrafoSimple not defined